
## step 1:- importing libraries

In [ ]:
import numpy as np
from imblearn.over_sampling import SMOTE
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn import metrics
from sklearn import preprocessing
from datetime import datetime
from sklearn import feature_selection
from sklearn import naive_bayes
from sklearn import tree
from sklearn import utils
from sklearn import ensemble
from sklearn import linear_model
from sklearn import neighbors
import random
import warnings
import xgboost
from scipy import stats
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
#ignoring future warning
warnings.simplefilter(action='ignore', category=FutureWarning)

## step 2:- importing dataset

In [ ]:
df_p=pd.read_csv("../input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv")
df_p=utils.shuffle(df_p,random_state=42)
df_p.info()

the dataset doenot have any null values

## step 3:-converting the output column into label

In [ ]:
df_p["op"]=(df_p["Attrition"]=="Yes").astype(np.int)
df_p.drop("Attrition",axis=1,inplace=True)

## step 4:-Separating categorical columns from continous columns
checking unique values of differnet columns

In [ ]:
for col in df_p.columns.values:
    print(str(col),df_p[str(col)].unique().shape[0])

dropping 4 columns which is not effecting the results

In [ ]:
l_drop=[]
for col in df_p.columns.values:
    if df_p[str(col)].unique().shape[0]==1:
        l_drop.append(str(col))

l_drop.append("EmployeeNumber")
df_p.drop(l_drop,axis=1,inplace=True)

taking string column names int0o one list

In [ ]:
l_str=[]
for col in df_p.columns.values:
    
    if type(df_p[str(col)][0])==str or df_p[str(col)].unique().shape[0]==1:
        l_str.append(str(col))
df_str=df_p[l_str].copy() 
df_str.columns.values

taking continous columns names into one list

In [ ]:
l_con=[]
for col in df_p.drop(l_str,axis=1).columns.values:
    if df_p[str(col)].unique().shape[0]>10:  
     l_con.append(str(col))
df_con=df_p[l_con].copy()  
l_con

taking numeric categorical columns names into one list

In [ ]:
l_cat=[]
for col in df_p.drop(l_str,axis=1).columns.values:
    if df_p[str(col)].unique().shape[0]<=10:  
     l_cat.append(str(col))
l_cat.remove("op")
df_cat=df_p[l_cat].copy()  
l_cat

## step 5:-Negative values and iqr test in continous columns

checking for negative values or null values

In [ ]:
for col in df_p[l_con].columns.values:
    if (df_p[str(col)]<=0).sum()>0:
        print("number of zeros or negative  in columns",str(col),(df_p[str(col)]<=0).sum())


In [ ]:
for col in df_p[l_con].columns.values:
    if (df_p[str(col)]<=0).sum()>0:
        print("number of zeros   in columns",str(col),(df_p[str(col)]==0).sum())

No negative values
Zeros are not anomolies

calculating IQR and identifying outliers

In [ ]:
def IQR(data):
    upper_quantile=data.quantile(0.75)
    lower_quantile=data.quantile(0.25)
    IQR=upper_quantile-lower_quantile
    outlier1=upper_quantile+1.5*IQR
    outlier2=lower_quantile-1.5*IQR
    return (IQR,outlier1,outlier2)

In [ ]:
for col in df_p[l_con].columns.values:
    i,outlier1,outlier2=IQR(df_p[str(col)])
    print("upper_outliers",df_p[df_p[str(col)]>outlier1].shape[0],"column name",str(col),"theroritical_max",outlier1,"max",df_p[str(col)].max())
    print("lower_outliers",df_p[df_p[str(col)]<outlier2].shape[0],"column name",str(col),"theoritical_min",outlier2,"min",df_p[str(col)].min())

upper outliers are present in MonthlyIncome,YearsWithCurrManager,YearsInCurrentRole,YearsAtCompany,YearsSinceLastPromotion columns


## Step 6:-Visualizing numerical columns
Heatmap

In [ ]:
fig,axis=plt.subplots(figsize=(50,50))
sns.heatmap(df_p[l_con+["op"]].corr(),annot=True,ax=axis)
plt.tight_layout()

Most important features

In [ ]:
l=[]
l+=list(df_p[l_con+["op"]].corr()["op"][df_p[l_con+["op"]].corr()["op"]>0].index)
l

Boxplot

In [ ]:
fig,axes=plt.subplots(nrows=3,ncols=4,figsize=(10,10))
axes1=axes.flatten()
index=0
for col in l_con:
    sns.boxplot(x="op",y=col,data=df_p,ax=axes1[index])
    index+=1
plt.tight_layout()

## Step 7:-Visualizing categorical data
countplot

In [ ]:
fig,axis=plt.subplots(figsize=(50,50))
sns.heatmap(df_p[l_cat+["op"]].corr(),annot=True,ax=axis)
plt.tight_layout()

In [ ]:
l+=list(df_p[l_cat+["op"]].corr()["op"][df_p[l_cat+["op"]].corr()["op"]>0].index)
l

In [ ]:
fig,axes=plt.subplots(nrows=4,ncols=2,figsize=(30,30))
axes1=axes.flatten()
index=0
for col in l_str:
    sns.countplot(y=col,hue="op",data=df_p,ax=axes1[index])
    index+=1
plt.tight_layout()

## Step 8:-Encoding catagorical columns

In [ ]:

df_p=pd.get_dummies(df_p,columns=l_str)


In [ ]:
df_p.columns.values

In [ ]:
lc=['BusinessTravel_Non-Travel',
       'BusinessTravel_Travel_Frequently', 'BusinessTravel_Travel_Rarely',
       'Department_Human Resources', 'Department_Research & Development',
       'Department_Sales', 'EducationField_Human Resources',
       'EducationField_Life Sciences', 'EducationField_Marketing',
       'EducationField_Medical', 'EducationField_Other',
       'EducationField_Technical Degree', 'Gender_Female', 'Gender_Male',
       'JobRole_Healthcare Representative', 'JobRole_Human Resources',
       'JobRole_Laboratory Technician', 'JobRole_Manager',
       'JobRole_Manufacturing Director', 'JobRole_Research Director',
       'JobRole_Research Scientist', 'JobRole_Sales Executive',
       'JobRole_Sales Representative', 'MaritalStatus_Divorced',
       'MaritalStatus_Married', 'MaritalStatus_Single', 'OverTime_No',
       'OverTime_Yes']
fig,axis=plt.subplots(figsize=(50,50))
sns.heatmap(df_p[lc+["op"]].corr(),annot=True,ax=axis)
plt.tight_layout()

In [ ]:
l+=list(df_p[lc+["op"]].corr()["op"][df_p[lc+["op"]].corr()["op"]>0].index)


In [ ]:
y=df_p.op
X=df_p.drop("op",axis=1)
sc = preprocessing.StandardScaler()
x=sc.fit_transform(X)

In [ ]:
xtrain,xtest,ytrain,ytest=model_selection.train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
sm=SMOTE(random_state=42,k_neighbors=2)
xtrainsm,ytrainsm=sm.fit_sample(xtrain,ytrain)

In [ ]:

lr = linear_model.LogisticRegression()
lr.fit(sc.transform(xtrainsm),ytrainsm)
testprediction=lr.predict(sc.transform(xtest))
print("Accuracy",metrics.accuracy_score(ytest,testprediction))
print("AUC",metrics.roc_auc_score(ytest,testprediction))
print("recall",metrics.recall_score(ytest,testprediction))
print(metrics.confusion_matrix(ytest,testprediction))
print(metrics.classification_report(ytest,testprediction))

In [ ]:
rf = RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
                            max_depth=8, max_features='auto', max_leaf_nodes=None,
                            min_impurity_decrease=0.0, min_impurity_split=None,
                            min_samples_leaf=2, min_samples_split=5,
                            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
                            oob_score=False, random_state=0, verbose=0, warm_start=False)
rf.fit(xtrainsm,ytrainsm)
testprediction=rf.predict(xtest)
trainprediction=rf.predict(xtrainsm)
print("Accuracy",metrics.accuracy_score(ytest,testprediction))
print("AUC",metrics.roc_auc_score(ytest,testprediction))
print("recall",metrics.recall_score(ytest,testprediction))
print("Accuracy train",metrics.accuracy_score(ytrainsm,trainprediction))
print("recall train",metrics.recall_score(ytrainsm,trainprediction))


naive byase

In [ ]:
nb=naive_bayes.GaussianNB()
nb.fit(xtrainsm,ytrainsm)
testprediction=nb.predict(xtest)
print("Accuracy",metrics.accuracy_score(ytest,testprediction))
print("AUC",metrics.roc_auc_score(ytest,testprediction))
print("recall",metrics.recall_score(ytest,testprediction))